# El Niño dataset - Topology of Time Series (v2)

In [24]:
# Import the gtda modules
from gtda.time_series import Resampler, SlidingWindow, takens_embedding_optimal_parameters, \
    TakensEmbedding, PermutationEntropy, SingleTakensEmbedding
from gtda.homology import WeakAlphaPersistence, VietorisRipsPersistence
from gtda.diagrams import Scaler, Filtering, PersistenceEntropy, BettiCurve, PairwiseDistance
from gtda.graphs import KNeighborsGraph, GraphGeodesicDistance

from gtda.pipeline import Pipeline

import numpy as np
from sklearn.metrics import pairwise_distances

import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
init_notebook_mode(connected=True)

# gtda plotting functions
from gtda.plotting import plot_heatmap
# Plotting functions
from gtda.plotting import plot_point_cloud

from sklearn.decomposition import PCA


## Data 

Data is publicly available here: http://www.meteo.psu.edu/holocene/public_html/supplements/MultiproxySpatial09/results/nino3all

In [25]:
pc_ENSO = np.loadtxt("/Users/jeannefernandez/Dropbox/Optimal_representatives_weather/data/ENSO/nino3all", skiprows=1)

In [26]:
fig = px.line( title='ENSO temperature estimation, between 500 and 2006')
fig.add_scatter(x = pc_ENSO[:, 0],y=pc_ENSO[:, 1], name='X')
fig.show()

## Fixed parameter analysis

In [31]:
x_fixed = pc_ENSO[:, 0]
y_fixed = pc_ENSO[:, 1]

In [76]:
embedding_dimension_fixed = 3       #arbitrary choice
embedding_time_delay_fixed = 3      #arbitrary choice
stride = 1                          #arbitrary choice

embedder_fixed = SingleTakensEmbedding(
    parameters_type="fixed",
    n_jobs=2,
    time_delay=embedding_time_delay_fixed,
    dimension=embedding_dimension_fixed,
    stride=stride,
)

In [77]:
y_fixed_embedded = embedder_fixed.fit_transform(y_fixed)
print(f"Shape of embedded time series: {y_fixed_embedded.shape}")

Shape of embedded time series: (1500, 3)


In [78]:
plot_point_cloud(y_fixed_embedded)

In [ ]:
#saving the pointcloud as 
np.savetxt('embedded_pointcloud_fixed.csv', y_fixed_embedded[0], delimiter=',')

## Non-fixed anylsis

In [42]:
x_nonfixed = pc_ENSO[:, 0]
y_nonfixed = pc_ENSO[:, 1]


In [65]:
max_embedding_time_delay_nonfixed = 100
max_embedding_dimension_nonfixed = 100
stride = 1
optimal_time_delay, optimal_embedding_dimension = takens_embedding_optimal_parameters(
    y_nonfixed, max_embedding_time_delay_nonfixed, max_embedding_dimension_nonfixed, stride=stride
    )

print(f"Optimal embedding time delay based on mutual information: {optimal_time_delay}")
print(f"Optimal embedding dimension based on false nearest neighbors: {optimal_embedding_dimension}")

Optimal embedding time delay based on mutual information: 9
Optimal embedding dimension based on false nearest neighbors: 6


In [73]:

embedder_nonfixed = SingleTakensEmbedding(
    parameters_type="fixed",
    n_jobs=2,
    time_delay=optimal_time_delay,
    dimension=optimal_embedding_dimension,
    stride=stride,
)

y_nonfixed_embedded = embedder_nonfixed.fit_transform(y_nonfixed)

plot_point_cloud(y_nonfixed_embedded)           #plots the 3 first dimensions 

In [62]:
#saving the pointcloud as 
np.savetxt('embedded_pointcloud_nonfixed.csv', y_nonfixed_embedded[0], delimiter=',')

## Persistence Diagrams

In [80]:
y_fixed_embedded = y_fixed_embedded[None, :, :]

# 0 - connected components, 1 - loops (2 - voids: to do later with more computational power)
homology_dimensions = [0, 1]# 2]

fixed_persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=6
)
print("Persistence diagram for fixed parameters")
fixed_persistence.fit_transform_plot(y_fixed_embedded)


Persistence diagram for fixed parameters


array([[[0.        , 0.0027828 , 0.        ],
        [0.        , 0.00560777, 0.        ],
        [0.        , 0.00563757, 0.        ],
        ...,
        [0.02618518, 0.03056262, 1.        ],
        [0.02609537, 0.02625548, 1.        ],
        [0.02493026, 0.02656609, 1.        ]]])

In [69]:
y_nonfixed_embedded = y_nonfixed_embedded[None, :, :]
nonfixed_persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=6
)
print("Persistence diagram for nonfixed parameters")
nonfixed_persistence.fit_transform_plot(y_nonfixed_embedded);

Persistence diagram for nonfixed parameters
